# Time Skew Testing

This animation system allows a user to define a frame and how long they would like that frame to be displayed for.
This only works if the ability to display frames is instantaneous. This, however, is not true. It takes time to loop through each pixel and set it's value. It also takes time to communicate those pixel values to the LED strand. This time should be measured and accounted for.

Imagine for a moment that we have an animation that is `FILL ME IN` frames long and that each frame is displayed for `FILL ME IN` ms. In a perfect system, this animation would display for 1`FILL ME IN` ms or `FILL ME IN` seconds, or `FILL ME IN`, which is considerably short. When we have a large number of short frames, it's possible that the communication latency is enought to become visibly noticlable. If we pretend that it takes 1 ms per render, this adds 10% to the length of our animation, which was not expected or accounted for, making it difficult to accurately time frames. Furthermore, this value changes from system to system.

The purpose of this notebook is to:
- Write a test for development purposed (or for those using this library what want to do their own testing).
- Experiment with ways to calculte these things.

## Different Timing Functions

At some point, we need a way to measure the passage of time. We want something that:
- Has high accuracy (ns would be nice)
- Is monotonic (it always goes forward; we don't care about time changes or TZs or daylight savings time)

The Python standard library has a few options that immediately come to mind:
- [time.monotonic_ns()](https://docs.python.org/3/library/time.html#time.monotonic_ns)
- [time.perf_counter_ns()](https://docs.python.org/3/library/time.html#time.perf_counter_ns)

Looking at those descriptions, the only noteable thing that is brought up is accounting for `sleep()`. Aside from that, there is no obvious reason to choose one over the other. Since `perf_counter_ns` seems to be named to imply that it's for measureing performance, I'll start with that one.

## Generic Testing Function

To run several different tests, we'll generate a generic testing function. This function will take:
- The number of pixels
- The number of frames
- The delay for each frame
- Something to send the pixels to

It will return multiple `List`s containing timings to be crunched later.

In [4]:
import sys


sys.path.append('../')

In [13]:
import random
import time

from adafruit_dotstar_pi_animation.data import *

def generate_random_color() -> int:
    return random.randrange(8) * 32


def generate_test_animation(pixel_count: int, frame_count: int, frame_delay: int) -> (int, Animation):
    """
    This function generates and times how long it takes to generate said test animation.
    """
    
    generation_start = time.perf_counter_ns()
    frames = []
    # For each frame:
    for f_idx in range(frame_count):
        pixels = []
        # for each pixel:
        for p_idx in range(pixel_count):
            pixel = Pixel(red=generate_random_color(), blue=generate_random_color(), green=generate_random_color(), brightness=1.0)
            pixels.append(pixel)
            
        frame = Frame(pixels=pixels, display_ms=frame_delay)
        frames.append(frame)
        
    animation = Animation(frames=frames, loop_infinitely=False, pause_between_play_ms=0, max_plays=1)
    generation_end = time.perf_counter_ns()
    
    return generation_end - generation_start, animation


def test_drawing_a(dot_star, animation: Animation) -> (List[int], List[int]):
    """
    This test handles instrumenting the "alpha" algorithm.
    """
    
    
    # Play Algo that we're testing
    ##############################
    parsing_ts = []
    tx_ts = []
    
    loop_count = 0
    while True:
        
        for frame_idx, frame in enumerate(animation.frames):
            # Loops the frames

            for pixel_idx, pixel in enumerate(frame.pixels):
                # Loops the pixels
                parsing_ts.append(time.perf_counter_ns())
                dot_star[pixel_idx] = (pixel.red, pixel.blue, pixel.green, pixel.brightness)
                parsing_ts.append(time.perf_counter_ns())

            # Show the frame for a while
            tx_ts.append(time.perf_counter_ns())
            dot_star.show()
            tx_ts.append(time.perf_counter_ns())
            
            time.sleep(frame.display_ms / 1000)

        loop_count += 1
        if loop_count >= animation.max_plays:
            break
            
    return parsing_ts, tx_ts
    

First test:

In [47]:
import adafruit_dotstar
import board


num_leds = 11
pixels = adafruit_dotstar.DotStar(board.SCLK, board.MOSI, num_leds, pixel_order=adafruit_dotstar.BGR, auto_write=False)
animation_generation_ns, animation = generate_test_animation(11, 10000, 10)
start_time = time.perf_counter_ns()
parse_timings, tx_timings = test_drawing_a(pixels, animation)
end_time = time.perf_counter_ns()

print(f'Total play time in ns: {end_time - start_time}')

Total play time in ns: 111698562877


In [48]:
def process_timing_pairs(timings):
    differences = []
    
    start_time = None
    for t in timings:
        if start_time is None:
            start_time = t
        else:
            differences.append(t - start_time)
            start_time = None
            
    return differences
        

parse_timing_differences = process_timing_pairs(parse_timings)
tx_timming_differences = process_timing_pairs(tx_timings)

Some basic numbers:
- How long did we expect the animation to take?
- How long did it actually take?

In [49]:
play_time_expected_seconds = animation.frame_total_time_ms / 1000
actual_play_time_seconds = (end_time - start_time) / 1000000000

play_time_expected_seconds, actual_play_time_seconds, abs(play_time_expected_seconds-actual_play_time_seconds)

(100.0, 111.698562877, 11.698562877)

Wow, that's 11.7 seconds slower. That's much larger than I was expecting. Let's see if we can completely account for that time.

In [50]:
from functools import reduce


time_lost_in_parsing = reduce(lambda x, y: x + y, parse_timing_differences)
time_lost_in_tx = reduce(lambda x, y: x + y, tx_timming_differences)

total_time_lost_ns = time_lost_in_parsing + time_lost_in_tx
total_time_lost_ns / 1000000000, time_lost_in_parsing / 1000000000, time_lost_in_tx / 1000000000

(10.096501132, 5.35875532, 4.737745812)

We can account for about 10 seconds of loss. Some of the additional loss might stem from:
- Our instrumentation efforts
- Inaccuracies in `time.sleep()`
- Other processes contending for time on this machine

TO be 1.7 seconds off over the course of 100 seconds isn't super significant, but it would be nice to study this further and to try to develop an algorithm that can correct and speed up or drop frames to catch up when it's slow.

What is the average time to parse all 11 pixels and to send them down the wire (in ms)?

In [51]:
import statistics


mean_parsing_time_per_pixel = statistics.mean(parse_timing_differences)
mean_tx_time_per_frame = statistics.mean(tx_timming_differences)

mean_parsing_time_per_pixel, mean_tx_time_per_frame, mean_tx_time_per_frame / num_leds

(48715.95745454545, 473774.5812, 43070.41647272727)

There is 0.04 ms delay for each pixel parsed and then an additional 0.04 ms delay for each one pushed down the wire. One would think these things scale linearly.

A 10 ms delay between frames is pretty high. That's 100hz, which is faster than the average TV. Let's lower this to 10hz and see if our numbers stay roughly the same.

In [89]:
animation_generation_ns, animation = generate_test_animation(11, 1000, 100)
start_time = time.perf_counter_ns()
parse_timings, tx_timings = test_drawing_a(pixels, animation)
end_time = time.perf_counter_ns()

print(f'Total play time in ns: {end_time - start_time}')

parse_timing_differences = process_timing_pairs(parse_timings)
tx_timming_differences = process_timing_pairs(tx_timings)
play_time_ns = end_time - start_time
expected_play_time_ns = animation.frame_total_time_ms * 1000000

time_lost_in_parsing = reduce(lambda x, y: x + y, parse_timing_differences)
time_lost_in_tx = reduce(lambda x, y: x + y, tx_timming_differences)
total_time_lost_ns = abs(play_time_ns - expected_play_time_ns)
print(f'Time lost in parsing: {time_lost_in_parsing/1000000000}')
print(f'Time lost in txt: {time_lost_in_tx/1000000000}')
print(f'Time lost total: {total_time_lost_ns/1000000000}')
print(f'Unaccounted lost time: {(total_time_lost_ns - time_lost_in_tx - time_lost_in_parsing)/1000000000}')


mean_parsing_time_per_pixel = statistics.mean(parse_timing_differences)
mean_tx_time_per_frame = statistics.mean(tx_timming_differences)
print(f'Mean parsing time per pixel: {mean_parsing_time_per_pixel}')
print(f'Mean tx time per pixel: {mean_tx_time_per_frame/num_leds}')

pixels.fill((0, 0, 0))
pixels.show()

Total play time in ns: 101397046881
Time lost in parsing: 0.584205988
Time lost in txt: 0.54583587
Time lost total: 1.397046881
Unaccounted lost time: 0.267005023
Mean parsing time per pixel: 53109.635272727275
Mean tx time per pixel: 49621.442727272726


Even with this reduced play rate, we lose a considerable amount of time (>1%), about 20% of which is not accounted for.

## Time Drift Correction

Imagine that at the start of every frame, we took note of the time. We could display each frame for the duration of `display_ms - parsing time - tx time`. This helps us, but does not account for `time.sleep()` also being inaccurate. We need to somehow track time drift across all frames. Another possible solution is to play the first frame, redcord it's total time, and display the next frame for that much less time. This allows the next frame to correct for the first. The third frame could do this for the first frame and so on. This has the advantage of tracking the drift at any point in time. It also gives the the ability to drop a frame if we know that we're `x` time behind and the next frame is `y` time where `x < y`. We could make this even more sophisticated by developing a quick (<10ms) way to test how long it takes to parse-n-push data to the LEDs, allowing us to take this into account, too. The hard part will be the math around dropping frames and resuming correct time.

In [146]:
def test_drawing_b(dot_star, animation: Animation, frames_per_adjustment=10, drift_threshold_ms=25) -> (List[int], List[int]):
    """
    This test handles instrumenting the "alpha" algorithm.
    """
    
    
    # Play Algo that we're testing
    ##############################
    total_drift_ns = 0
    
    loop_count = 0
    adjustments = []
    while True:
        
        last_frame_start = time.perf_counter_ns()
        for frame_idx, frame in enumerate(animation.frames):
            # Loops the frames
            
            show_adjustment_ns = 0
            
            for pixel_idx, pixel in enumerate(frame.pixels):
                # Loops the pixels
                dot_star[pixel_idx] = (pixel.red, pixel.blue, pixel.green, pixel.brightness)

            # Show the frame for a while
            dot_star.show()
            
            # Do we need to adjust with some sort of correction?
            if frame_idx % frames_per_adjustment == 0:
                # Check for the need to correct
                if total_drift_ns > drift_threshold_ms * 1000000:
                    show_adjustment_ns = min(max(total_drift_ns / 2, frame.display_ms * 1000000 / 2), frame.display_ms * 1000000)
            # Account for analysis after
            adjustments.append((frame_idx, total_drift_ns, show_adjustment_ns))
                
            # Let the frame stand for a bit
            time_to_show_sec = (frame.display_ms / 1000) - (show_adjustment_ns / 1000000000)
            time.sleep(time_to_show_sec)
            
            # Use a third value to allow timers to overlap
            end_show = time.perf_counter_ns()
            current_frame_start = end_show
            
            # Drif accounting
            actual_display_time_ns = end_show - last_frame_start
            frame_display_ns = frame.display_ms * 1000000
            frame_drift_ns = actual_display_time_ns - frame_display_ns
            total_drift_ns = total_drift_ns + frame_drift_ns
            
            last_frame_start = current_frame_start

        loop_count += 1
        if loop_count >= animation.max_plays:
            break
            
    return adjustments

In [147]:
animation_generation_ns, animation = generate_test_animation(11, 1000, 100)
start_time = time.perf_counter_ns()
adjustments = test_drawing_b(pixels, animation)
end_time = time.perf_counter_ns()

print(f'Total play time in ns: {end_time - start_time}')

Total play time in ns: 100026607927


In [148]:
adjustments

[(0, 0, 0),
 (1, 925547, 0),
 (2, 1877395, 0),
 (3, 2676329, 0),
 (4, 3413701, 0),
 (5, 4284770, 0),
 (6, 5206202, 0),
 (7, 6104354, 0),
 (8, 6831049, 0),
 (9, 7536859, 0),
 (10, 8336991, 0),
 (11, 9201809, 0),
 (12, 10598600, 0),
 (13, 11809821, 0),
 (14, 12877137, 0),
 (15, 14153565, 0),
 (16, 15129371, 0),
 (17, 16416997, 0),
 (18, 17533062, 0),
 (19, 18589493, 0),
 (20, 19590768, 0),
 (21, 20732927, 0),
 (22, 21836701, 0),
 (23, 22875423, 0),
 (24, 23980812, 0),
 (25, 25000055, 0),
 (26, 26084142, 0),
 (27, 27088438, 0),
 (28, 28186170, 0),
 (29, 29191508, 0),
 (30, 30275126, 50000000.0),
 (31, -18801872, 0),
 (32, -17782004, 0),
 (33, -16552033, 0),
 (34, -15608257, 0),
 (35, -14463338, 0),
 (36, -13525760, 0),
 (37, -12513496, 0),
 (38, -11327327, 0),
 (39, -10053816, 0),
 (40, -8794887, 0),
 (41, -7541376, 0),
 (42, -6235052, 0),
 (43, -4952479, 0),
 (44, -3654332, 0),
 (45, -2367279, 0),
 (46, -1025332, 0),
 (47, 429480, 0),
 (48, 2101371, 0),
 (49, 3242697, 0),
 (50, 4580791, 

In [149]:
pixels.fill((0, 0, 0))
pixels.show()